### 네이버 뉴스 크롤링 최종
- 섹션 추출 ~ sub 페이지 추출

#### (1) + (2) + (3)
(1) 섹션 메뉴와 섹션별 url 추출      
    - section, link  
    - section_menu_df  
(2) 섹션별 헤드라인(topic) 제목과 url 추출  
    - topic, url, section  
    - all_topic_df  
(3) 섹션 헤드라인별 sub 페이지 기사 내용 추출  
    - 언론사, 제목, 날짜/시간, 기사내용, 작성자  
    - paper, title, datetime, writing, writer  
    - all_sub_news_info_df   

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [2]:
from urllib.request import urlopen
import pandas as pd
import bs4
import requests

In [3]:
url = 'https://news.naver.com/'
html = urlopen(url)
# 파서 객체 생성
bs_obj = bs4.BeautifulSoup(html, 'html.parser')

#### (1) 네이버 뉴스 섹션 메뉴와 url 추출

In [4]:
ul = bs_obj.find('ul', {'class':'Nlnb_menu_list'})
# ul

In [5]:
# 수집된 데이터로 df 구성

lis = ul.select('li')

section = []
link = []

# 리스트에 추가 
for li in lis:
    a_tag = li.find('a')
    section.append(a_tag.text)
    link.append(a_tag['href'])

section_menu_df = pd.DataFrame({'section':section, 'link':link})
section_menu_df

,section,link
0,언론사별,https://news.naver.com/?viewType=pc
1,정치,https://news.naver.com/main/main.naver?mode=LS...
2,경제,https://news.naver.com/main/main.naver?mode=LS...
3,사회,https://news.naver.com/main/main.naver?mode=LS...
4,생활/문화,https://news.naver.com/main/main.naver?mode=LS...
5,IT/과학,https://news.naver.com/main/main.naver?mode=LS...
6,세계,https://news.naver.com/main/main.naver?mode=LS...
7,랭킹,https://news.naver.com/main/ranking/popularDay...
8,신문보기,https://news.naver.com/newspaper/home?viewType=pc
9,오피니언,https://news.naver.com/opinion/home


#### (2) 섹션별 헤드라인(topic) 제목과 url 추출

In [6]:
# 해당 섹션에 대한 모든 topic 크롤링하는 함수 생성
def get_topic(url, section):
    # 접속 및 결과 파싱
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}
    res = requests.get(url, headers=headers)
    html = res.text
    bs_obj = bs4.BeautifulSoup(html, 'html.parser')

    # topic_list 추출
    topic_list = bs_obj.findAll('div', {'class':'sh_text'})

    # 제목과 링크 추출해서 리스트에 추가
    topic_title = []
    topic_link = []

    for topic in topic_list:    
        try :
            a = topic.find('a')
            topic_title.append(a.text)
            topic_link.append(a['href'])
        except:
            print('오류 발생')
            # pass

    # 딕셔너리 만들어서 반환
    return ({'topic':topic_title, 'url':topic_link, 'section':section})

In [7]:
# 모든 섹션에 대한 모든 topic 데이터프레임 생성
all_topic_df = pd.DataFrame({
    'topic':[],
    'url': [],
    'section': []
})

for i in range(1, 7) :
    df = pd.DataFrame(get_topic(section_menu_df['link'][i],  section_menu_df['section'][i]))
    all_topic_df = pd.concat([all_topic_df, df], axis=0, ignore_index=True)

# all_topic_df

#### (3) 섹션 헤드라인별 sub 페이지 기사 내용 추출
- 언론사, 제목, 날짜/시간, 기사내용, 작성자

In [8]:
# sub 페이지 기사 내용 추출하는 함수
def get_sub_news_info(url):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}
    # 빈 리스트 생성
    paper_list, title_list, datetime_list, writing_list, writer_list = [ [] for _ in range(5) ]
    # for _ in range(5) : 언더바(_) (변수 없이 반복만 수행)
    
    try :
        
        res = requests.get(url, headers=headers)
        html = res.text
        bs_obj = bs4.BeautifulSoup(html, 'html.parser')
        # 접속    
    
        # 언론사, 기사제목, 작성 날짜 시간, 기사 내용, 작성자 리스트 생성
        paper_list.append(bs_obj.find('a', {'class':'media_end_head_top_logo'}).select_one('img')['title'])
        title_list.append(bs_obj.find('div', {'class' : 'media_end_head_title'}).select_one('span').text)
        datetime_list.append(bs_obj.find('span', {'class' : 'media_end_head_info_datestamp_time _ARTICLE_DATE_TIME'}).text)
        writing_list.append(bs_obj.select_one('#dic_area').text.replace('\n', ''))
        writer_list.append(bs_obj.find('span',{'class':'byline_s'}).text)

    except:
        print("오류 발생")
        # pass


    # 딕셔너리로 만들어서 반환
    sub_news_dict = {
        'paper':paper_list,
        'title':title_list,
        'datetime':datetime_list,
        'writing':writing_list,
        'writer':writer_list
    }

    return sub_news_dict

In [9]:
# 모든 sub 페이지 내용 저장하는 데이터프레임 생성해서 최종 데이터프레임 생성
all_sub_news_info_df = pd.DataFrame({
    'paper': [],
    'title': [],
    'datetime': [],
    'writing': [],
    'writer': []
})

for url in all_topic_df['url']:
    df = pd.DataFrame(get_sub_news_info(url))
    all_sub_news_info_df = pd.concat([all_sub_news_info_df, df], axis=0, ignore_index=True)

all_sub_news_info_df

,paper,title,datetime,writing,writer
0,중앙일보,"김정은 ""4년만에 방러, 북러관계 전략적 중요성 뚜렷이 표현""",2023.09.13. 오전 6:08,\t\t\t 김정은 북한 국무위원장이 12일 오전 러시아 국경도시 하산에 도착했다...,정혜정 기자 jeong.hyejeong@joongang.co.kr
1,부산일보,"김기현, 자우림 김윤아 겨냥 ""개념 없는 개념연예인 많아""",2023.09.12. 오후 5:36,국민의힘 김기현 대표가 12일 오후 서울 중구 한국관광공사에서 열린 사단법인 문화자...,성규환 기자(bastion@busan.com)
2,뉴시스,"AI사업단, 서울대 AI연구원·AI양재허브 등과 맞손",2023.09.13. 오후 2:13,초거대 AI모델·서비스 개발 위한 컴퓨팅자원 확대 지원AI 관련 신규 사업·우수 기...,송창헌 기자(goodchang@newsis.com)
3,세계일보,우럭·멍게·전복 등…매주 수요일은 공무원 ‘수산물 먹는 날’,2023.09.12. 오후 2:16,세종·서울·과천·대전 정부청사 구내식당 17곳서 중식 제공‘수(水)요일엔 우리 수(...,김경호 기자 stillcut@segye.com
4,디지털타임스,"""北서 온 쓰레기"" 폭언 항의 눈감던 이재명…태영호 ""단식걱정했더니 뒷말 조롱에 징계?""",2023.09.12. 오후 3:51,"野 ""단식장서 행패"" 징계 엄포…太 ""'쓰레기, 빨갱이' 막말에 악마화 꼼수뿐""""눈...",한기호 기자(hkh89@dt.co.kr)
5,뉴스1,"홍준표 ""내가 독고다이? 같이 일하는 훌륭한 사람 주위에 많아""",2023.09.13. 오전 10:48,"""내가 특이한게 아니라 한국 정치가 비정상""홍준표 대구시장이 17일 오후 서울 여의...",남승렬 기자 (pdnamsy@news1.kr)
6,세계일보,“건강 먼저” 당내 단식 중단 요구에도…‘단식 14일차’ 이재명 “아직 똘똘. 윤 ...,2023.09.13. 오후 1:12,"이 대표, 단식 계속 이어가겠다는 의사민주당 의원들, 이 대표의 건강 악화 우려 단...",김경호 기자 stillcut@segye.com
7,kbc광주방송,"이재명 ""尹, 국방장관 교체로 꼬리 자르기..특검으로 진상규명""",2023.09.13. 오전 10:54,▲검찰 조사 마치고 나오는 이재명 대표 사진 : 연합뉴스더불어민주당 이재명 대표가 ...,구영슬(seulnews@ikbc.co.kr)
8,YTN,"與, '허위 인터뷰 의혹' 방송한 김어준 등 3명 고발키로",2023.09.13. 오후 2:18,\t\t\t국민의힘은 대장동 사건 핵심 인물인 김만배 씨 허위 인터뷰 의혹과 관련해...,이종원(jongwon@ytn.co.kr)
9,MBN,"박지현 ""이재명 보자마자 그냥 눈물 났다…자연스러운 일""",2023.09.13. 오전 10:44,"""이전의 이념이나 생각 변한 것 아냐""더불어민주당 이재명 대표가 지난 11일 국회 ...",윤혜주 디지털뉴스 기자 heyjude@mbn.co.kr


In [10]:
# 최종 결과 파일로 저장
all_sub_news_info_df.to_csv('./crawl_data/naver_sub_news_section.csv', index=0)

In [11]:
all_sub_news_info_df = pd.read_csv('./crawl_data/naver_sub_news_section.csv')
all_sub_news_info_df

,paper,title,datetime,writing,writer
0,중앙일보,"김정은 ""4년만에 방러, 북러관계 전략적 중요성 뚜렷이 표현""",2023.09.13. 오전 6:08,\t\t\t 김정은 북한 국무위원장이 12일 오전 러시아 국경도시 하산에 도착했다...,정혜정 기자 jeong.hyejeong@joongang.co.kr
1,부산일보,"김기현, 자우림 김윤아 겨냥 ""개념 없는 개념연예인 많아""",2023.09.12. 오후 5:36,국민의힘 김기현 대표가 12일 오후 서울 중구 한국관광공사에서 열린 사단법인 문화자...,성규환 기자(bastion@busan.com)
2,뉴시스,"AI사업단, 서울대 AI연구원·AI양재허브 등과 맞손",2023.09.13. 오후 2:13,초거대 AI모델·서비스 개발 위한 컴퓨팅자원 확대 지원AI 관련 신규 사업·우수 기...,송창헌 기자(goodchang@newsis.com)
3,세계일보,우럭·멍게·전복 등…매주 수요일은 공무원 ‘수산물 먹는 날’,2023.09.12. 오후 2:16,세종·서울·과천·대전 정부청사 구내식당 17곳서 중식 제공‘수(水)요일엔 우리 수(...,김경호 기자 stillcut@segye.com
4,디지털타임스,"""北서 온 쓰레기"" 폭언 항의 눈감던 이재명…태영호 ""단식걱정했더니 뒷말 조롱에 징계?""",2023.09.12. 오후 3:51,"野 ""단식장서 행패"" 징계 엄포…太 ""'쓰레기, 빨갱이' 막말에 악마화 꼼수뿐""""눈...",한기호 기자(hkh89@dt.co.kr)
5,뉴스1,"홍준표 ""내가 독고다이? 같이 일하는 훌륭한 사람 주위에 많아""",2023.09.13. 오전 10:48,"""내가 특이한게 아니라 한국 정치가 비정상""홍준표 대구시장이 17일 오후 서울 여의...",남승렬 기자 (pdnamsy@news1.kr)
6,세계일보,“건강 먼저” 당내 단식 중단 요구에도…‘단식 14일차’ 이재명 “아직 똘똘. 윤 ...,2023.09.13. 오후 1:12,"이 대표, 단식 계속 이어가겠다는 의사민주당 의원들, 이 대표의 건강 악화 우려 단...",김경호 기자 stillcut@segye.com
7,kbc광주방송,"이재명 ""尹, 국방장관 교체로 꼬리 자르기..특검으로 진상규명""",2023.09.13. 오전 10:54,▲검찰 조사 마치고 나오는 이재명 대표 사진 : 연합뉴스더불어민주당 이재명 대표가 ...,구영슬(seulnews@ikbc.co.kr)
8,YTN,"與, '허위 인터뷰 의혹' 방송한 김어준 등 3명 고발키로",2023.09.13. 오후 2:18,\t\t\t국민의힘은 대장동 사건 핵심 인물인 김만배 씨 허위 인터뷰 의혹과 관련해...,이종원(jongwon@ytn.co.kr)
9,MBN,"박지현 ""이재명 보자마자 그냥 눈물 났다…자연스러운 일""",2023.09.13. 오전 10:44,"""이전의 이념이나 생각 변한 것 아냐""더불어민주당 이재명 대표가 지난 11일 국회 ...",윤혜주 디지털뉴스 기자 heyjude@mbn.co.kr
